In [ ]:
# =============================================================================
# SETUP AND IMPORTS
# =============================================================================
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import logging
import sys
import warnings
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Project paths
repo_root = Path.cwd().parent
src_path = repo_root / "src"
scripts_path = repo_root / "scripts"
data_path = repo_root / "data"
results_path = repo_root / "results"

# Add paths
sys.path.insert(0, str(src_path))
sys.path.insert(0, str(scripts_path))

# Check Gurobi availability
try:
    import gurobipy as gp
    GUROBI_AVAILABLE = True
    print(f"✓ Gurobi version: {gp.gurobi.version()}")
except ImportError:
    GUROBI_AVAILABLE = False
    print("⚠ Gurobi not available - will fall back to HiGHS or other solver")

print(f"\nPyPSA version: {pypsa.__version__}")
print(f"Project root: {repo_root}")

## 1. Load Network

Load the network we built in network_05.ipynb or create a fresh one.

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Network configuration
N_CLUSTERS = 200
STRATEGY = 'kmeans'
START_DATE = '2023-01-01'
END_DATE = '2023-01-07'  # One week for faster optimization

# Solver configuration
SOLVER_NAME = 'gurobi' if GUROBI_AVAILABLE else 'highs'

# Gurobi solver options (following PyPSA-EUR)
GUROBI_OPTIONS = {
    'MIPGap': 0.01,           # 1% optimality gap
    'TimeLimit': 7200,         # 2 hour time limit
    'Threads': 4,              # Number of threads
    'Method': 2,               # Barrier method (2) for LP
    'Crossover': 0,            # No crossover (faster)
    'BarHomogeneous': 1,       # Homogeneous algorithm
    'OutputFlag': 1,           # Show solver output
}

# HiGHS fallback options
HIGHS_OPTIONS = {
    'threads': 4,
    'time_limit': 7200,
}

SOLVER_OPTIONS = GUROBI_OPTIONS if SOLVER_NAME == 'gurobi' else HIGHS_OPTIONS

print(f"Solver: {SOLVER_NAME}")
print(f"Period: {START_DATE} to {END_DATE}")

In [ ]:
# =============================================================================
# LOAD OR BUILD NETWORK
# =============================================================================

# Try to load pre-built network from network_05
network_file = data_path / "networks" / "clustered" / f"network_{N_CLUSTERS}_complete.nc"

if network_file.exists():
    print(f"Loading network from {network_file}...")
    n = pypsa.Network(str(network_file))
else:
    print("Building network from scratch...")
    
    # Load clustered network base
    cluster_path = data_path / "networks" / "clustered" / f"network_clustered_{STRATEGY}_{N_CLUSTERS}.nc"
    
    if cluster_path.exists():
        n = pypsa.Network(str(cluster_path))
    else:
        raise FileNotFoundError(f"Network not found. Please run network_04.ipynb first.")
    
    # Set snapshots
    snapshots = pd.date_range(START_DATE, END_DATE, freq='h')
    n.set_snapshots(snapshots)
    
    # Add generators using our module
    from add_generators import add_all_generators
    n = add_all_generators(n, project_root=repo_root, aggregate_renewables=True)

print(f"\nNetwork summary:")
print(f"  Buses: {len(n.buses)}")
print(f"  Lines: {len(n.lines)}")
print(f"  Generators: {len(n.generators)}")
print(f"  Loads: {len(n.loads)}")
print(f"  Snapshots: {len(n.snapshots)}")

## 2. Network Validation Before Optimization

Critical checks before running the optimizer.

In [ ]:
# =============================================================================
# VALIDATE NETWORK
# =============================================================================

def validate_network_for_optimization(n):
    """Check network is ready for optimization."""
    issues = []
    
    # Check basics
    if len(n.buses) == 0:
        issues.append("❌ No buses")
    if len(n.generators) == 0:
        issues.append("❌ No generators")
    if len(n.loads) == 0:
        issues.append("❌ No loads")
    if len(n.snapshots) == 0:
        issues.append("❌ No snapshots")
    
    # Check capacity adequacy
    total_capacity = n.generators['p_nom'].sum() / 1000  # GW
    
    if len(n.loads_t.p_set.columns) > 0:
        peak_demand = n.loads_t.p_set.sum(axis=1).max() / 1000  # GW
    else:
        peak_demand = n.loads['p_set'].sum() / 1000
    
    margin = (total_capacity / peak_demand - 1) * 100
    
    if margin < 10:
        issues.append(f"⚠ Low capacity margin: {margin:.1f}%")
    
    # Check line capacities
    if (n.lines['s_nom'] <= 0).any():
        issues.append("⚠ Some lines have zero/negative capacity")
    
    # Check generator constraints
    if (n.generators['p_nom'] < 0).any():
        issues.append("❌ Negative generator capacities")
    
    # Report
    print("="*60)
    print("NETWORK VALIDATION")
    print("="*60)
    print(f"\nCapacity: {total_capacity:.1f} GW")
    print(f"Peak demand: {peak_demand:.1f} GW")
    print(f"Margin: {margin:.1f}%")
    
    if issues:
        print(f"\n⚠ Issues found:")
        for issue in issues:
            print(f"  {issue}")
    else:
        print("\n✓ Network ready for optimization")
    
    return len(issues) == 0

is_valid = validate_network_for_optimization(n)

## 3. Configure Optimization Problem

### 3.1 Dispatch Optimization (Economic Dispatch)

Minimize operational costs with fixed capacities.

In [ ]:
# =============================================================================
# PREPARE FOR DISPATCH OPTIMIZATION
# =============================================================================

# Make a copy for dispatch optimization
n_dispatch = n.copy()

# Ensure all capacities are fixed (not extendable)
n_dispatch.generators['p_nom_extendable'] = False
n_dispatch.lines['s_nom_extendable'] = False
if len(n_dispatch.storage_units) > 0:
    n_dispatch.storage_units['p_nom_extendable'] = False

print("Dispatch optimization configuration:")
print(f"  Extendable generators: {n_dispatch.generators['p_nom_extendable'].sum()}")
print(f"  Extendable lines: {n_dispatch.lines['s_nom_extendable'].sum()}")
print(f"  Solver: {SOLVER_NAME}")

In [ ]:
# =============================================================================
# RUN DISPATCH OPTIMIZATION
# =============================================================================

print("\n" + "="*60)
print("SOLVING DISPATCH OPTIMIZATION")
print("="*60)
print(f"\nUsing solver: {SOLVER_NAME}")
print(f"Snapshots: {len(n_dispatch.snapshots)}")
print(f"Variables (approx): {len(n_dispatch.generators) * len(n_dispatch.snapshots)}")

# Suppress gurobi logging during solve
if SOLVER_NAME == 'gurobi':
    logging.getLogger('gurobipy').setLevel(logging.WARNING)

# Solve
status, termination_condition = n_dispatch.optimize(
    solver_name=SOLVER_NAME,
    solver_options=SOLVER_OPTIONS,
)

print(f"\n{'='*60}")
print(f"Status: {status}")
print(f"Termination: {termination_condition}")
print(f"Objective value: {n_dispatch.objective / 1e6:.2f} M€")

In [ ]:
# =============================================================================
# ANALYZE DISPATCH RESULTS
# =============================================================================

def analyze_dispatch_results(n, title="Dispatch Results"):
    """Analyze optimization results."""
    print("\n" + "="*60)
    print(title)
    print("="*60)
    
    # System cost
    print(f"\nTotal system cost: {n.objective / 1e6:.2f} M€")
    
    # Generation by carrier
    if 'p' in dir(n.generators_t) and len(n.generators_t.p.columns) > 0:
        gen_by_carrier = n.generators_t.p.groupby(
            n.generators.carrier, axis=1
        ).sum().sum() / 1e6  # TWh
        
        print(f"\nGeneration by carrier (TWh):")
        for carrier, gen in gen_by_carrier.sort_values(ascending=False).items():
            print(f"  {carrier:15} {gen:8.2f}")
        
        total_gen = gen_by_carrier.sum()
        print(f"  {'TOTAL':15} {total_gen:8.2f}")
    
    # Load served
    if len(n.loads_t.p.columns) > 0:
        total_load = n.loads_t.p.sum().sum() / 1e6
        print(f"\nTotal load served: {total_load:.2f} TWh")
    
    # Line congestion
    if 'p0' in dir(n.lines_t) and len(n.lines_t.p0.columns) > 0:
        line_loading = n.lines_t.p0.abs() / n.lines['s_nom']
        congested = (line_loading > 0.95).any().sum()
        print(f"\nCongested lines (>95%): {congested}/{len(n.lines)}")
    
    # Marginal prices
    if 'marginal_price' in dir(n.buses_t) and len(n.buses_t.marginal_price.columns) > 0:
        avg_price = n.buses_t.marginal_price.mean().mean()
        max_price = n.buses_t.marginal_price.max().max()
        print(f"\nMarginal prices:")
        print(f"  Average: {avg_price:.2f} €/MWh")
        print(f"  Maximum: {max_price:.2f} €/MWh")

analyze_dispatch_results(n_dispatch)

### 3.2 Visualize Dispatch Results

In [ ]:
# =============================================================================
# PLOT DISPATCH STACK
# =============================================================================

# Color scheme (following PyPSA-EUR convention)
CARRIER_COLORS = {
    'wind': '#3B6B8C',
    'solar': '#F9D423',
    'hydro': '#4A90D9',
    'gas': '#E74C3C',
    'hard coal': '#2C3E50',
    'lignite': '#8B4513',
    'nuclear': '#9B59B6',
    'oil': '#95A5A6',
    'biomass': '#27AE60',
    'biogas': '#2ECC71',
    'waste': '#7F8C8D',
    'geothermal': '#E67E22',
    'other': '#BDC3C7',
}

if len(n_dispatch.generators_t.p.columns) > 0:
    fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    
    # Generation stack
    ax1 = axes[0]
    gen_by_carrier = n_dispatch.generators_t.p.groupby(
        n_dispatch.generators.carrier, axis=1
    ).sum() / 1000  # GW
    
    # Sort by average generation
    gen_order = gen_by_carrier.mean().sort_values(ascending=False).index
    colors = [CARRIER_COLORS.get(c, '#BDC3C7') for c in gen_order]
    
    gen_by_carrier[gen_order].plot.area(
        ax=ax1, color=colors, alpha=0.8, linewidth=0
    )
    
    # Add load line
    total_load = n_dispatch.loads_t.p.sum(axis=1) / 1000
    ax1.plot(total_load.index, total_load.values, 'k--', linewidth=2, label='Load')
    
    ax1.set_ylabel('Power (GW)')
    ax1.set_title('Generation Dispatch by Carrier')
    ax1.legend(loc='upper right', ncol=4)
    ax1.set_ylim(0, None)
    ax1.grid(True, alpha=0.3)
    
    # Marginal prices
    ax2 = axes[1]
    if len(n_dispatch.buses_t.marginal_price.columns) > 0:
        avg_price = n_dispatch.buses_t.marginal_price.mean(axis=1)
        ax2.plot(avg_price.index, avg_price.values, 'b-', linewidth=1)
        ax2.fill_between(avg_price.index, 0, avg_price.values, alpha=0.3)
        ax2.set_ylabel('Price (€/MWh)')
        ax2.set_title('Average Marginal Price')
        ax2.grid(True, alpha=0.3)
    
    plt.xlabel('Time')
    plt.tight_layout()
    plt.show()
else:
    print("No dispatch results available")

In [ ]:
# =============================================================================
# PLOT GENERATION MIX
# =============================================================================

if len(n_dispatch.generators_t.p.columns) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Pie chart of total generation
    ax1 = axes[0]
    gen_total = n_dispatch.generators_t.p.groupby(
        n_dispatch.generators.carrier, axis=1
    ).sum().sum()
    gen_total = gen_total[gen_total > 0].sort_values(ascending=False)
    
    colors = [CARRIER_COLORS.get(c, '#BDC3C7') for c in gen_total.index]
    ax1.pie(gen_total, labels=gen_total.index, colors=colors, autopct='%1.1f%%')
    ax1.set_title('Generation Mix')
    
    # Capacity factor by carrier
    ax2 = axes[1]
    cf_by_carrier = []
    for carrier in gen_total.index:
        carrier_gens = n_dispatch.generators[n_dispatch.generators.carrier == carrier].index
        if len(carrier_gens) > 0:
            generation = n_dispatch.generators_t.p[carrier_gens].sum().sum()
            capacity = n_dispatch.generators.loc[carrier_gens, 'p_nom'].sum()
            cf = generation / (capacity * len(n_dispatch.snapshots)) * 100
            cf_by_carrier.append({'carrier': carrier, 'cf': cf})
    
    cf_df = pd.DataFrame(cf_by_carrier).set_index('carrier')
    colors = [CARRIER_COLORS.get(c, '#BDC3C7') for c in cf_df.index]
    cf_df['cf'].plot.barh(ax=ax2, color=colors)
    ax2.set_xlabel('Capacity Factor (%)')
    ax2.set_title('Capacity Factors by Carrier')
    ax2.set_xlim(0, 100)
    ax2.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.show()

## 4. Capacity Expansion Optimization

Now we allow investments in new generation and transmission capacity.

In [ ]:
# =============================================================================
# CONFIGURE CAPACITY EXPANSION
# =============================================================================

# Make a copy for expansion planning
n_expand = n.copy()

# Capital costs for new capacity (€/MW/year, following PyPSA-EUR)
CAPITAL_COSTS = {
    'wind': 85000,       # Onshore wind
    'solar': 40000,      # Utility-scale solar PV
    'gas': 45000,        # CCGT
    'battery': 150000,   # Li-ion battery
}

# Maximum expansion potential (MW per bus)
MAX_EXPANSION = {
    'wind': 10000,   # 10 GW per bus
    'solar': 10000,  # 10 GW per bus
    'gas': 5000,     # 5 GW per bus
}

# Set extendable carriers
EXTENDABLE_CARRIERS = ['wind', 'solar', 'gas']

# Make selected carriers extendable
for carrier in EXTENDABLE_CARRIERS:
    mask = n_expand.generators.carrier == carrier
    n_expand.generators.loc[mask, 'p_nom_extendable'] = True
    n_expand.generators.loc[mask, 'capital_cost'] = CAPITAL_COSTS.get(carrier, 0)
    n_expand.generators.loc[mask, 'p_nom_max'] = (
        n_expand.generators.loc[mask, 'p_nom'] + MAX_EXPANSION.get(carrier, 10000)
    )

# Also allow line expansion
n_expand.lines['s_nom_extendable'] = True
n_expand.lines['s_nom_max'] = n_expand.lines['s_nom'] * 3  # Max 3x current capacity
n_expand.lines['capital_cost'] = 400  # €/MW/km (simplified)

print("Capacity expansion configuration:")
print(f"  Extendable carriers: {EXTENDABLE_CARRIERS}")
print(f"  Extendable generators: {n_expand.generators['p_nom_extendable'].sum()}")
print(f"  Extendable lines: {n_expand.lines['s_nom_extendable'].sum()}")

In [ ]:
# =============================================================================
# RUN CAPACITY EXPANSION OPTIMIZATION
# =============================================================================

print("\n" + "="*60)
print("SOLVING CAPACITY EXPANSION")
print("="*60)
print(f"\nUsing solver: {SOLVER_NAME}")

# More aggressive solver settings for expansion problem
expand_solver_options = SOLVER_OPTIONS.copy()
if SOLVER_NAME == 'gurobi':
    expand_solver_options['MIPGap'] = 0.05  # 5% gap for faster solve

# Solve
status, termination_condition = n_expand.optimize(
    solver_name=SOLVER_NAME,
    solver_options=expand_solver_options,
)

print(f"\n{'='*60}")
print(f"Status: {status}")
print(f"Termination: {termination_condition}")
print(f"Objective value: {n_expand.objective / 1e6:.2f} M€")

In [ ]:
# =============================================================================
# ANALYZE EXPANSION RESULTS
# =============================================================================

def analyze_expansion_results(n):
    """Analyze capacity expansion results."""
    print("\n" + "="*60)
    print("CAPACITY EXPANSION RESULTS")
    print("="*60)
    
    # New generator capacity by carrier
    new_gen_cap = n.generators.groupby('carrier').apply(
        lambda g: (g['p_nom_opt'] - g['p_nom']).sum() / 1000  # GW
    )
    new_gen_cap = new_gen_cap[new_gen_cap.abs() > 0.01]
    
    print(f"\nNew generation capacity (GW):")
    for carrier, cap in new_gen_cap.sort_values(ascending=False).items():
        print(f"  {carrier:15} {cap:+8.2f}")
    print(f"  {'TOTAL':15} {new_gen_cap.sum():+8.2f}")
    
    # New line capacity
    if 's_nom_opt' in n.lines.columns:
        new_line_cap = (n.lines['s_nom_opt'] - n.lines['s_nom']).sum() / 1000  # GW
        print(f"\nNew transmission capacity: {new_line_cap:+.2f} GW")
    
    # Investment costs breakdown
    print(f"\nInvestment costs:")
    
    gen_investment = n.generators.groupby('carrier').apply(
        lambda g: ((g['p_nom_opt'] - g['p_nom']).clip(lower=0) * g['capital_cost']).sum() / 1e6
    )
    gen_investment = gen_investment[gen_investment > 0.01]
    
    for carrier, cost in gen_investment.sort_values(ascending=False).items():
        print(f"  {carrier:15} {cost:8.2f} M€")
    
    total_gen_investment = gen_investment.sum()
    print(f"  {'Total gen':15} {total_gen_investment:8.2f} M€")

analyze_expansion_results(n_expand)

In [ ]:
# =============================================================================
# COMPARE DISPATCH VS EXPANSION
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Capacity comparison
ax1 = axes[0]

cap_dispatch = n_dispatch.generators.groupby('carrier')['p_nom'].sum() / 1000
cap_expand = n_expand.generators.groupby('carrier')['p_nom_opt'].sum() / 1000

cap_compare = pd.DataFrame({
    'Before': cap_dispatch,
    'After': cap_expand
}).fillna(0)

cap_compare = cap_compare[cap_compare.max(axis=1) > 1]  # Filter small
cap_compare.plot.barh(ax=ax1)
ax1.set_xlabel('Capacity (GW)')
ax1.set_title('Capacity: Before vs After Expansion')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='x')

# Cost comparison
ax2 = axes[1]

costs = pd.Series({
    'Dispatch Only': n_dispatch.objective / 1e6,
    'With Expansion': n_expand.objective / 1e6
})

bars = costs.plot.bar(ax=ax2, color=['#3498db', '#27ae60'])
ax2.set_ylabel('Total Cost (M€)')
ax2.set_title('System Cost Comparison')
ax2.tick_params(axis='x', rotation=0)
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels
for i, v in enumerate(costs.values):
    ax2.text(i, v + costs.max()*0.02, f'{v:.1f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 5. Advanced: Adding Constraints

Following PyPSA-EUR, we can add custom constraints like CO₂ limits.

In [ ]:
# =============================================================================
# ADD CO2 CONSTRAINT
# =============================================================================

# CO2 emission factors (t CO2 / MWh)
CO2_FACTORS = {
    'gas': 0.41,
    'hard coal': 0.90,
    'lignite': 1.10,
    'oil': 0.70,
    'biomass': 0.0,   # Considered carbon-neutral
    'biogas': 0.0,
    'wind': 0.0,
    'solar': 0.0,
    'nuclear': 0.0,
    'hydro': 0.0,
}

# Calculate emissions from dispatch optimization
if len(n_dispatch.generators_t.p.columns) > 0:
    emissions_by_carrier = {}
    total_emissions = 0
    
    for carrier, factor in CO2_FACTORS.items():
        carrier_gens = n_dispatch.generators[
            n_dispatch.generators.carrier == carrier
        ].index
        
        if len(carrier_gens) > 0 and any(g in n_dispatch.generators_t.p.columns for g in carrier_gens):
            valid_gens = [g for g in carrier_gens if g in n_dispatch.generators_t.p.columns]
            generation = n_dispatch.generators_t.p[valid_gens].sum().sum()  # MWh
            emissions = generation * factor / 1e6  # Mt CO2
            emissions_by_carrier[carrier] = emissions
            total_emissions += emissions
    
    print("\nCO2 Emissions from Dispatch Optimization:")
    print("="*40)
    for carrier, emissions in sorted(emissions_by_carrier.items(), key=lambda x: -x[1]):
        if emissions > 0.001:
            print(f"  {carrier:15} {emissions:.3f} Mt CO2")
    print(f"  {'TOTAL':15} {total_emissions:.3f} Mt CO2")

In [ ]:
# =============================================================================
# SOLVE WITH CO2 LIMIT (USING GLOBAL CONSTRAINT)
# =============================================================================

# Create network with CO2 limit
n_co2 = n.copy()

# Make renewables extendable
for carrier in ['wind', 'solar']:
    mask = n_co2.generators.carrier == carrier
    n_co2.generators.loc[mask, 'p_nom_extendable'] = True
    n_co2.generators.loc[mask, 'capital_cost'] = CAPITAL_COSTS.get(carrier, 0)
    n_co2.generators.loc[mask, 'p_nom_max'] = (
        n_co2.generators.loc[mask, 'p_nom'] + 50000  # 50 GW additional
    )

# Add CO2 carriers attribute to generators
for gen_idx, gen in n_co2.generators.iterrows():
    carrier = gen['carrier']
    co2_factor = CO2_FACTORS.get(carrier, 0)
    # PyPSA uses efficiency to calculate emissions
    # emissions = generation / efficiency * co2_factor
    # For simplicity, we encode directly

# Add carriers with CO2 emissions
carriers_df = pd.DataFrame({
    'co2_emissions': [CO2_FACTORS.get(c, 0) for c in n_co2.generators.carrier.unique()]
}, index=n_co2.generators.carrier.unique())

# Add to network carriers
for carrier in carriers_df.index:
    if carrier not in n_co2.carriers.index:
        n_co2.add('Carrier', carrier, co2_emissions=carriers_df.loc[carrier, 'co2_emissions'])
    else:
        n_co2.carriers.loc[carrier, 'co2_emissions'] = carriers_df.loc[carrier, 'co2_emissions']

# Set CO2 limit (50% reduction from baseline)
CO2_LIMIT = total_emissions * 0.5 * 1e6  # Convert back to tonnes

# Add global CO2 constraint
n_co2.add(
    'GlobalConstraint',
    'co2_limit',
    type='primary_energy',
    carrier_attribute='co2_emissions',
    sense='<=',
    constant=CO2_LIMIT
)

print(f"\nSolving with CO2 limit: {CO2_LIMIT/1e6:.3f} Mt (50% reduction)")

# Solve
status, termination_condition = n_co2.optimize(
    solver_name=SOLVER_NAME,
    solver_options=expand_solver_options,
)

print(f"\nStatus: {status}")
print(f"Objective: {n_co2.objective / 1e6:.2f} M€")

## 6. Save Optimized Network

Export the solved network for further analysis.

In [ ]:
# =============================================================================
# SAVE RESULTS
# =============================================================================

results_dir = results_path / "optimization"
results_dir.mkdir(parents=True, exist_ok=True)

# Save optimized networks
dispatch_file = results_dir / "network_dispatch_optimized.nc"
expand_file = results_dir / "network_expansion_optimized.nc"

n_dispatch.export_to_netcdf(dispatch_file)
n_expand.export_to_netcdf(expand_file)

print(f"Saved dispatch results to: {dispatch_file}")
print(f"Saved expansion results to: {expand_file}")

# Export summary statistics
summary = {
    'dispatch_cost_meur': n_dispatch.objective / 1e6,
    'expansion_cost_meur': n_expand.objective / 1e6,
    'dispatch_gen_twh': n_dispatch.generators_t.p.sum().sum() / 1e6,
    'n_buses': len(n.buses),
    'n_generators': len(n.generators),
    'n_snapshots': len(n.snapshots),
}

summary_df = pd.Series(summary)
summary_df.to_csv(results_dir / "optimization_summary.csv")

print("\nSummary:")
print(summary_df)

## Summary

This notebook demonstrated:

1. **Dispatch optimization** - Economic dispatch with fixed capacities
2. **Capacity expansion** - Joint optimization of dispatch and investments
3. **CO₂ constraints** - Adding emission limits using PyPSA's GlobalConstraint
4. **Result analysis** - Generation mix, costs, capacity factors, congestion

### Key PyPSA-EUR Methodology Elements Used:

- **Gurobi solver** with appropriate options (barrier method, MIP gap)
- **Carrier-based modeling** for generation technologies
- **Capital costs** for extendable components
- **CO₂ emission factors** by carrier
- **Global constraints** for policy limits

### Next Steps:

- Add storage (batteries, pumped hydro)
- Multi-period optimization with rolling horizon
- Sector coupling (heating, transport)
- Sensitivity analysis with scenarios

---

*Following PyPSA-EUR methodology for European energy system modeling.*